In [1]:
"""
Created on Mon May 13 11:45:31 2024

The goal of this script is to use one dicom file to map a region to will then be used by medtool to create mesh

@author: cofo
"""
import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
from PIL import Image
import scipy
import os
import shutil
import pandas as pd

import sys
# Add the directory to the sys.path
sys.path.append('D:/EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis')
# Now you can import modules from that directory
import tools.CT_modification_functions as CT_modification_functions
import tools.CT_visualization_functions as CT_visualization_functions
%matplotlib qt

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

print(current_dir)
print(parent_dir)


direction_map_whole_solid = current_dir + '\map_whole_meshing_region.mhd'

direction_map_keratine = current_dir + '\map_keratine_region.mhd'

output_folder_path = current_dir


d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup\031 - create mesh volume\python
d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup\031 - create mesh volume


<>:8: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
<>:8: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
C:\Users\cofo\AppData\Local\Temp\ipykernel_3752\180928903.py:8: SyntaxWarning: invalid escape sequence '\m'
  direction_map_whole_solid = current_dir + '\map_whole_meshing_region.mhd'
C:\Users\cofo\AppData\Local\Temp\ipykernel_3752\180928903.py:10: SyntaxWarning: invalid escape sequence '\m'
  direction_map_keratine = current_dir + '\map_keratine_region.mhd'


In [3]:
# Load the keratine zone from file

map_keratine = sitk.ReadImage(direction_map_keratine) 

image_keratine = CT_modification_functions.dilate_filter(map_keratine,1)

# Load the image from file
map_whole_solid = sitk.ReadImage(direction_map_whole_solid)

# Convert the SimpleITK image to a NumPy array
solid_array = sitk.GetArrayFromImage(map_whole_solid)
keratine_array = sitk.GetArrayFromImage(map_keratine)

#see it


CT_visualization_functions.browse_blended_images(map_whole_solid, map_keratine,True)



D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:199: RuntimeWarning: invalid value encountered in divide
  image2 = image2 / np.max(image2) * 255
D:\EmpaData/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis\tools\CT_visualization_functions.py:203: RuntimeWarning: invalid value encountered in cast
  blended_image = np.clip(blended_image, 0, 255).astype(np.uint8)


In [5]:
result_image = sitk.Subtract(map_whole_solid, map_keratine)
result_image = CT_modification_functions.apply_threshold(result_image,50,255)

# Coordinates of the specific pixel
x, y, z = 313,293,1488
#x, y, z = 0,0,0
#print(binary_image[x, y, z])

# Use the connected component filter with the seed 
connected_filter = sitk.ConnectedThresholdImageFilter()
connected_filter.SetLower(1)
connected_filter.SetUpper(255)
connected_filter.SetReplaceValue(255)
connected_filter.SetSeedList([(x, y, z)])


result_image = CT_modification_functions.erode_filter(result_image,1)
result_image = connected_filter.Execute(result_image)
result_image = CT_modification_functions.dilate_filter(result_image,1)

result_image = sitk.Cast(result_image, sitk.sitkUInt8)

#see it
result_array = sitk.GetArrayFromImage(result_image)
CT_visualization_functions.browse_images(result_image)
CT_visualization_functions.browse_blended_images(map_whole_solid, result_image,True)



In [6]:

sitk.WriteImage(result_image, output_folder_path + "/map_bone_region.mhd")
print(output_folder_path)

d:\EmpaData\COFO - HP\01 PhD research\202406 - Basel skulls\WP2 - skull 9254\03 - Simulation Setup\031 - create mesh volume\python
